In [1]:
import psycopg2
import random
import string
import time
from tqdm import tqdm

In [2]:
# Database Information
DATABASE_NAME = "users_test1"
DATABASE_USERNAME = ""
DATABASE_PASSWORD = ""
DATABASE_HOST = "localhost"
DATABASE_POST = "5432"

# NOTE: Please ensure DATABASE_NAME (users_test1 in this case) database exists in the Postgres server

# Establishing the connection
conn = psycopg2.connect(
   database=DATABASE_NAME, user=DATABASE_USERNAME, password=DATABASE_PASSWORD, host=DATABASE_HOST, port=DATABASE_POST
)

# Creating a cursor object using the cursor() method
cursor = conn.cursor()

In [3]:
# Droping EMPLOYEE table if already exists.
cursor.execute("DROP TABLE IF EXISTS EMPLOYEE")

# Creating table as per requirement
sql ='''CREATE TABLE EMPLOYEE(
   FIRST_NAME CHAR(20) NOT NULL,
   LAST_NAME CHAR(20),
   AGE INT,
   SEX CHAR(1),
   INCOME FLOAT
)'''

cursor.execute(sql)
conn.commit()

print("Table created successfully")

Table created successfully


In [4]:
# Preparing SQL queries to INSERT a record into the database.
for i in tqdm(range(0, 500000)):
    sql = f'''
        INSERT INTO EMPLOYEE(
            FIRST_NAME, 
            LAST_NAME, 
            AGE, 
            SEX,
            INCOME
        ) VALUES (
            '{''.join(random.choices(string.ascii_uppercase + string.ascii_lowercase, k=5))}', 
            '{''.join(random.choices(string.ascii_uppercase + string.ascii_lowercase, k=5))}', 
            {random.randint(22, 58)}, 
            '{random.choice(['F','M'])}', 
            {random.randint(100000, 1000000)}
        )
    '''
    cursor.execute(sql)

# Commit your changes in the database
conn.commit()

print("Records inserted")

100%|██████████| 500000/500000 [00:23<00:00, 21566.21it/s]

Records inserted


# Testing Index on age column

In [5]:
# Querying without index
sql = '''
    EXPLAIN ANALYZE SELECT * FROM EMPLOYEE WHERE age = 34;
'''
cursor.execute(sql)
cursor.fetchall()

[('Gather  (cost=1000.00..10646.17 rows=13600 width=56) (actual time=0.297..19.756 rows=13447 loops=1)',),
 ('  Workers Planned: 2',),
 ('  Workers Launched: 2',),
 ('  ->  Parallel Seq Scan on employee  (cost=0.00..8286.17 rows=5667 width=56) (actual time=0.016..14.034 rows=4482 loops=3)',),
 ('        Filter: (age = 34)',),
 ('        Rows Removed by Filter: 162184',),
 ('Planning Time: 0.181 ms',),
 ('Execution Time: 20.289 ms',)]

In [10]:
# Create an index on age column
sql = '''
    CREATE INDEX IDX_EMPLOYEE_AGE 
    ON EMPLOYEE(age);
'''
cursor.execute(sql)
conn.commit()

In [11]:
# Querying with index
sql = '''
    EXPLAIN ANALYZE SELECT * FROM EMPLOYEE WHERE age = 34;
'''
cursor.execute(sql)
cursor.fetchall()

[('Bitmap Heap Scan on employee  (cost=257.82..6109.82 rows=13600 width=56) (actual time=1.289..6.261 rows=13447 loops=1)',),
 ('  Recheck Cond: (age = 34)',),
 ('  Heap Blocks: exact=5188',),
 ('  ->  Bitmap Index Scan on idx_employee_age  (cost=0.00..254.42 rows=13600 width=0) (actual time=0.820..0.821 rows=13447 loops=1)',),
 ('        Index Cond: (age = 34)',),
 ('Planning Time: 0.178 ms',),
 ('Execution Time: 6.547 ms',)]